In [11]:
import numpy as np
import pandas as pd
import cv2
from pathlib import Path
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.utils import shuffle

# --- Config ---
MODEL_PATH = "../results/model_weights/final_model.h5"
LABEL_CSV = "../data/cibs-ddsm/metadata/labels_resolved.csv"
SAVE_DIR = Path("../results/gradcam_images")
SAVE_DIR.mkdir(parents=True, exist_ok=True)
IMG_SIZE = (224, 224)
LAST_CONV_LAYER = "conv2d_1"

# --- Load Model and get inner Sequential ---
full_model = load_model(MODEL_PATH)
model = full_model.get_layer("sequential")
model(tf.zeros((1, 224, 224, 1)))  # call once to build the graph

# --- Grad-CAM ---
def make_gradcam_heatmap(img_array, model, last_conv_layer_name):
    grad_model = tf.keras.models.Model(
        inputs=model.input,
        outputs=[model.get_layer(last_conv_layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        class_channel = predictions[:, 0]

    grads = tape.gradient(class_channel, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]

    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

# --- Image Preprocessing ---
def preprocess_image(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, IMG_SIZE)
    img = img.astype(np.float32) / 255.0
    img = np.expand_dims(img, axis=-1)
    return np.expand_dims(img, axis=0)

# --- Overlay ---
def overlay_heatmap(img_path, heatmap, alpha=0.4, colormap=cv2.COLORMAP_JET):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, IMG_SIZE)
    img_color = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    heatmap = cv2.resize(heatmap, IMG_SIZE)
    heatmap = np.uint8(255 * heatmap)
    heatmap_color = cv2.applyColorMap(heatmap, colormap)
    return cv2.addWeighted(heatmap_color, alpha, img_color, 1 - alpha, 0)

# --- Load Metadata and Samples ---
df = pd.read_csv(LABEL_CSV)
df = shuffle(df, random_state=42).reset_index(drop=True)
sample_df = pd.concat([
    df[df["label"] == 0].sample(2, random_state=1),
    df[df["label"] == 1].sample(2, random_state=1)
])

# --- Generate Grad-CAMs ---
for i, row in sample_df.iterrows():
    img_path = row["image_file_path"]
    label = "Malignant" if row["label"] == 1 else "Benign"

    img_tensor = preprocess_image(img_path)
    heatmap = make_gradcam_heatmap(img_tensor, model, LAST_CONV_LAYER)
    overlay = overlay_heatmap(img_path, heatmap)

    save_path = SAVE_DIR / f"{label}_{Path(img_path).stem}_gradcam.png"
    cv2.imwrite(str(save_path), overlay)

    plt.figure(figsize=(4, 4))
    plt.imshow(overlay[..., ::-1])
    plt.title(label)
    plt.axis("off")
    plt.show()

ValueError: No such layer: sequential. Existing layers are: ['random_flip', 'random_rotation', 'random_zoom', 'conv2d', 'max_pooling2d', 'conv2d_1', 'max_pooling2d_1', 'flatten', 'dense', 'dropout', 'dense_1'].